In [ ]:
import json
import pprint

In [ ]:
with open('./dataset/recipe0928_V5.json', 'r', encoding='utf-8') as f:
    content = json.loads(f.read())

In [ ]:
#content[:5]

In [ ]:
total_i_list = []
for each_recipe in content:
    each_i_list =[]
    for each_i in each_recipe['ingredient']:
        each_i_list.append(each_i[0])
    for each_s in each_recipe['seasoning']:
        each_i_list.append(each_s[0])
    #print(each_i_list)
    total_i_list.append(each_i_list)
total_i_list

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

In [ ]:
#Transform data to pandas DataFrame as apriori function requests (to only TRUE/FALSE for higher effeiciency)
te = TransactionEncoder()
te_ary = te.fit(total_i_list).transform(total_i_list)
te_ary = te_ary.astype("int")

In [ ]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df

In [ ]:
from sklearn import preprocessing
'''
preprocessing.scale()是按照列进行标准化计算，计算公式为:
(X_train[:,0]-X_train[:,0].mean())/X_train[:,0].std()
(X_train[:,0]-np.mean(X_train[:,0]))/np.std(X_train[:,0])//或者
'''
X_scaled = preprocessing.scale(df)  # scale操作之后的数据零均值，单位方差（方差为1）
X_scaled[0:5]

In [ ]:
# 进行PCA数据降维
from sklearn.decomposition import PCA
 
# 生成PCA实例
#pca = PCA(n_components=296) # 把维度降至1/3 -296维
pca = PCA(n_components=90) 
# 进行PCA降维
X_pca = pca.fit_transform(X_scaled)

# 生成降维后的dataframe
X_pca_frame = pd.DataFrame(X_pca)  
X_pca_frame

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

In [ ]:
silhouette_avg = []
for i in range(2,11):
    clf = KMeans(n_clusters=i)
    kmeans_fit = clf.fit(X_pca_frame)
    silhouette_avg.append(silhouette_score(X_pca_frame, kmeans_fit.labels_))

In [ ]:
plt.plot(range(2,11), silhouette_avg)

In [ ]:
clf = KMeans(n_clusters=4)
kmeans_fit = clf.fit(X_pca_frame)
print(kmeans_fit.labels_)
kmeans_clustering_labels = pd.DataFrame(kmeans_fit.labels_, columns=['cluster'])
print(kmeans_clustering_labels)

In [ ]:
recipe_name_list = []
for each_recipe in content:
    recipe_name_list.append(each_recipe['recipe'])

recipe_name = pd.Series(recipe_name_list)
recipe_name

In [ ]:
recipe_cluster = pd.concat([recipe_name,df, kmeans_clustering_labels], axis=1)
recipe_cluster

In [ ]:
recipe_cluster[recipe_cluster['cluster']== 0]  #取出cluster為0的食譜

In [ ]:
item_name = list(recipe_cluster.columns)  #889種食材名
del item_name[0]
del item_name[-1]

sum_in_cluster = [(recipe_cluster[recipe_cluster['cluster']== 0].iloc[:,num]).sum() for num in range(1,890)]  #每種食材出現次數總和(cluster=0時)
sum_in_cluster
len(sum_in_cluster)

In [ ]:
len(item_name)

In [ ]:
#方法二: 手刻文字雲需要的input-字串 (同等以下『' '.join(all)』)

_str = ''
for i in range(676):
    count = sum_in_cluster[i]
    for j in range(count):
        _str += item_name[i] + ' '

In [ ]:
#方法一
#step1: 從dataframe抓出食譜名及所屬cluster
cluster = list(recipe_cluster['cluster'])
re_name = list(recipe_cluster[0])

In [ ]:
#step2:製作食譜及所屬cluster字典
cluster_dic = dict()  
for each in range(len(cluster)):
    cluster_dic[re_name[each]] = cluster[each]
cluster_dic

In [ ]:
cluster_catagories = [{},{},{},{}]
for k in range(4):
    a = cluster_catagories[k]
    a[k] = []
print(cluster_catagories)
    
    

for r,c in cluster_dic.items():
    for k in range(4):
        if c == k:
            cluster_catagories[k][k].append(r)
cluster_catagories   

for i in range(4):
    with open('./output/kmeans_with_seasoning_recipe_lower/cluster%s.txt'%(i),'w', encoding='utf-8') as f:
        f.write(str(cluster_catagories[i][i]))

In [ ]:
import pprint

In [ ]:
#step3:將cluster加進原食譜
for i in content:    
    if i['recipe'] in list(cluster_dic.keys()):
        #print(i['recipe'])
        i['cluster_num'] = cluster_dic[i['recipe']]
#pprint.pprint(content[:10])

In [ ]:
#step4:抓出指定cluster中的食材做成串列
all = []
for i in content:
    if i['cluster_num'] == 0:
        for each_item in i['ingredient']:
            all.append(each_item[0])
           

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def plot_wordcloud(text, title=None, max = 1000, size=(12,8), title_size=16):
    """plots wordcloud"""
    wordcloud = WordCloud(max_words=max,font_path='‪C:\Windows\Fonts\kaiu.ttf',collocations=False).generate(text)
    plt.figure(figsize=size)
    plt.title(title, size=title_size)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")

plot_wordcloud(' '.join(all), title='cluster_0_Ingredients')

In [ ]:
#方法三
#製作詞頻字典(成為文字雲generate_from_frequency的input)
count_item = dict()
for i in range(889):
    count_item[item_name[i]] = int(sum_in_cluster[i])
count_item

In [ ]:
from wordcloud import WordCloud
wordcloud_try = WordCloud(font_path='‪C:\Windows\Fonts\kaiu.ttf').generate_from_frequencies(count_item)
plt.imshow(wordcloud_try, interpolation='bilinear')

In [ ]:
dic_list = [{},{},{},{},{}]
for k in range(5):
    sum_in_cluster_single = [(recipe_cluster[recipe_cluster['cluster']== k].iloc[:,num]).sum() for num in range(1,890)]
    for i in range(675):
        dic_list[k][item_name[i]] = int(sum_in_cluster_single[i])
dic_list

In [ ]:
wordcloud_0 = WordCloud(font_path='‪C:\Windows\Fonts\kaiu.ttf').generate_from_frequencies(dic_list[0])
plt.imshow(wordcloud_0, interpolation='bilinear')

In [ ]:
wordcloud_1 = WordCloud(font_path='‪C:\Windows\Fonts\kaiu.ttf').generate_from_frequencies(dic_list[1])
plt.imshow(wordcloud_1, interpolation='bilinear')

In [ ]:
wordcloud_2 = WordCloud(font_path='‪C:\Windows\Fonts\kaiu.ttf').generate_from_frequencies(dic_list[2])
plt.imshow(wordcloud_2, interpolation='bilinear')

In [ ]:
wordcloud_3 = WordCloud(font_path='‪C:\Windows\Fonts\kaiu.ttf').generate_from_frequencies(dic_list[3])
plt.imshow(wordcloud_3, interpolation='bilinear')

In [ ]:
wordcloud_4 = WordCloud(font_path='‪C:\Windows\Fonts\kaiu.ttf').generate_from_frequencies(dic_list[4])
plt.imshow(wordcloud_4, interpolation='bilinear')